# Retrieving Dataset from `yfinance` python API

The Tickers we would be using mainly to extract the financial data price of the futures are the following from Yahoo Finance:
- `CL=F` -> Priority (West Texas Intermediate Barrel):
- `GC=F` -> Priority (Gold Futures):

We would be using `yf.Ticker` and see the columns, we want to retrieve the following in order to create a **Regression models** for days: 
* `Ticker:` Market quotation symbol for the future, in our case would be `CL=F, GC=F, H0=F,` and many more that can retrieve the prices of gold and oil futures.
* `Commodity:` Name of the asset being analyzed, we have basically two: oil and gold. This would be key when partitioning in the AWS Athena for Statistical arbitrage | Momentum Trading.
* `Date:` The date the data was recorded. Format `YYYY-MM-DD`.
* `Open:` Market opening price.
* `High:` Highest price during the trading day
* `Low:` Lowest price during the day.
* `Close:` Market Closing Price.
* `Volume:` Number of contracts traded during the day

In [2]:
import yfinance as yf

In [9]:
# Let's create the different tickets ffor 
future_tickers = ["CL=F", "GC=F"]

def get_stock_data_test(ticker:str) -> dict:
    '''
    Function to retrieve the stock dataset for the specific two tickers.
    '''
    stock = yf.Ticker(ticker=ticker)
    return stock.info

In [7]:
# West Texas Intermediate Barrel
west_stock_barrel_futures = get_stock_data_test(future_tickers[0])

In [8]:
west_stock_barrel_futures

{'maxAge': 86400,
 'priceHint': 2,
 'previousClose': 61.07,
 'open': 59.66,
 'dayLow': 59.52,
 'dayHigh': 61.36,
 'regularMarketPreviousClose': 61.07,
 'regularMarketOpen': 59.66,
 'regularMarketDayLow': 59.52,
 'regularMarketDayHigh': 61.36,
 'volume': 260445,
 'regularMarketVolume': 260445,
 'averageVolume': 245465,
 'averageVolume10days': 295373,
 'averageDailyVolume10Day': 295373,
 'bid': 61.28,
 'ask': 61.33,
 'expireDate': 1771545600,
 'openInterest': 372999,
 'fiftyTwoWeekLow': 54.98,
 'fiftyTwoWeekHigh': 78.4,
 'allTimeHigh': 147.27,
 'allTimeLow': -40.32,
 'fiftyDayAverage': 58.5998,
 'twoHundredDayAverage': 62.2496,
 'currency': 'USD',
 'tradeable': False,
 'quoteType': 'FUTURE',
 'symbol': 'CL=F',
 'language': 'en-US',
 'region': 'US',
 'typeDisp': 'Futures',
 'quoteSourceName': 'Delayed Quote',
 'triggerable': False,
 'customPriceAlertConfidence': 'NONE',
 'headSymbolAsString': 'CL=F',
 'contractSymbol': False,
 'marketState': 'REGULAR',
 'corporateActions': [],
 'regularMa